In [ ]:
!pip install -U -r requirements.txt

## 학습 전 모델로 MBTI 분류해보기

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'EleutherAI/polyglot-ko-1.3b' #1.3b, 3.8b, 5.8b, 12.8b

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL,
    device=0
)

In [ ]:
def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

    ans = pipe(
        conversation_history + "\n\n### 답변:",
        do_sample=True,
        max_new_tokens=64,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )

    msg = ans[0]["generated_text"]

    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    return state, state_chatbot, state_chatbot, msg

In [ ]:
state = [
           {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
    ]

state_chatbot = []

In [ ]:
while True:
  text = input("질문: ")

  if text == '그만':
    break

  state, state_chatbot, _, msg = answer(state, state_chatbot,text)
  print("답변: ", msg)


## MBTI 질문 데이터로 모델 학습 시키기

###!!!!이 코드 실행 전 런타임 다시 시작 눌러주세요!!!!

In [ ]:
!python run_clm.py \
--model_name_or_path='EleutherAI/polyglot-ko-1.3b' \
--train_file='./mbti_0920_small.json' \
--num_train_epochs=4 \
--block_size=256 \
--per_device_train_batch_size=1 \
--gradient_accumulation_steps=8 \
--fp16 \
--output_dir='my_model' \
--do_train \
--optim='adafactor' \
--learning_rate='2e-5' \
--logging_strategy='steps' \
--logging_first_step \
--run_name='polyglot-1.3b-koalpaca-v1.1a-rtx3090' \
--low_cpu_mem_usage

## 학습된 모델로 질문해보기

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = '/content/my_model'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL,
    device=0
)

In [ ]:
def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    # print('messages: ', messages)
    # print('state: ', state)
    # print('text:', text)

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

    ans = pipe(
        conversation_history + "\n\n### 답변:",
        do_sample=True,
        max_new_tokens=64,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )

    msg = ans[0]["generated_text"]

    # print("msg: ", msg)

    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    # print(state)
    # print(state_chatbot)

    return state, state_chatbot, msg

In [ ]:
state = [
            {
                "role": "맥락",
                "content": "MBTI 분류 챗봇입니다.",
            },
            {
                "role": "명령어",
                "content": "사람들의 답변을 기반으로 MBTI를 맞힙니다. "},
    ]

state_chatbot = []

In [ ]:
while True:
  text = input("질문: ")

  if text == '그만':
    break

  state, state_chatbot, msg = answer(state, state_chatbot,text)
  print("답변: ", msg)


## 학습한 모델 실제 챗봇 서비스로 실행시키기

In [ ]:
def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

    ans = pipe(
        conversation_history + "\n\n### 답변:",
        do_sample=True,
        max_new_tokens=64,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )

    msg = ans[0]["generated_text"]

    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    print(state)
    print(state_chatbot)

    return state, state_chatbot, state_chatbot


In [ ]:
import gradio as gr

with gr.Blocks(css="#chatbot .overflow-y-auto{height:750px}") as demo:
    state = gr.State(
        [
           {
                "role": "맥락",
                "content": "MBTI 분류 챗봇입니다.",
            },
            {
                "role": "명령어",
                "content": "사람들의 답변을 기반으로 MBTI를 맞힙니다. "},
        ]
    )
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML(
            """<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>나만의 MBTI 분류기</h1>
            </div>
            <div>
                Demo runs on RTX 3090 (24GB) with 8bit quantized
            </div>
        </div>"""
        )

    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Send a message...")

    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "", None, txt)

demo.queue().launch(debug=True, server_name="0.0.0.0", share=True)